In [32]:
import numpy as np
import pymc as pm
from sklearn.preprocessing import scale
from scipy import stats
from IPython.core.debugger import set_trace

In [62]:
Y0 = 57.0 + scale(np.random.normal(size=80)) * 5.42; 
Y0.mean(), Y0.std()

(57.0, 5.420000000000001)

In [18]:
with pm.Model() as m0:
    mu = pm.Normal('mu', mu=50, sigma=10)
    sigma = pm.Uniform('sigma', 0, 10)
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=Y0)
    idata = pm.sample(5000, tune=5000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, sigma]


Sampling 4 chains for 5_000 tune and 5_000 draw iterations (20_000 + 20_000 draws total) took 17 seconds.


In [19]:
idata

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

In [48]:
def from_posterior(param, samples, k=100):
    smin, smax = np.min(samples), np.max(samples)
    width = smax - smin
    x = np.linspace(smin, smax, k)
    y = stats.gaussian_kde(samples)(x)
    
    # what was never sampled should have a small probability but not 0,
    # so we'll extend the domain and use linear approximation of density on it
    x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]])
    y = np.concatenate([[0], y, [0]])
    #set_trace()
    return pm.Interpolated(param, x_points=x, pdf_points=y)

In [49]:
idata.posterior['mu'].mean(dim=['chain'])

<xarray.DataArray 'mu' (draw: 5000)>
array([57.58941348, 56.93850082, 57.23112421, ..., 56.75000843,
       57.51639167, 57.66271215])
Coordinates:
  * draw     (draw) int32 0 1 2 3 4 5 6 7 ... 4993 4994 4995 4996 4997 4998 4999

In [67]:
Y1 = np.random.normal(loc=57, scale=5.42, size=1)
Y1.mean(), Y1.std()

(52.027074021626284, 0.0)

In [73]:
with pm.Model() as m1:
    mu = from_posterior('mu', idata.posterior['mu'].mean(dim=['chain']))
    sigma = from_posterior('sigma', idata.posterior['sigma'].mean(dim=['chain']))
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=Y1)
    idata1 = pm.sample(50, tune=50, cores=8, chains=2)

Only 50 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 8 jobs)
NUTS: [mu, sigma]


Sampling 2 chains for 50 tune and 50 draw iterations (100 + 100 draws total) took 5 seconds.
The number of samples is too small to check convergence reliably.


In [69]:
idata1

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data